In [105]:
import pandas as pd

pd.set_option('display.max.rows', 500)
pd.set_option('display.max.columns', 25)

#reads rankings from human fantasy exerts
df = pd.read_csv("Rankings2022.csv", index_col="PLAYER NAME", usecols= ["RK","PLAYER NAME", "TEAM", "POS"])

#reads fantasy per game stats from selected years stores them in a dictionary
years = range(2019,2022)
stats = {year: pd.read_csv("Stats" + str(year) + ".csv", index_col="PLAYER NAME").fillna(0) for year in years}

#stat categories recorded by the data frame
categories = ["G","GS","PassYds","PassTD","Int","RushAtt","RushYds","Y/A","RushTD","Tgt","Rec","RecYds","RecTD","FL","TD","PPR","VBD"]

#compiling all data into aggregate data 
total = stats[2020].merge(stats[2021], on="PLAYER NAME", how="outer", suffixes= ("_2020", "_2021")).fillna(0)

#multi year per game averages 
for category in categories:
    total[category] = (total['G_2020'] * total[category + "_2020"] + total['G_2021'] * total[category + "_2021"]) / (total['G_2020'] + total['G_2021'])

#reformat age data 
total["Age"] = total["Age_2021"]

#creats sub data frames for each position
rankings = {}
for position in ["RB", "WR", "QB", "TE"]:
    filter = df["POS"].str.contains(position)
    rankings[position] = df.loc[filter]


def get_stats(year = 'ALL', position=""):
    global df

    #determines if specific year is wanted or aggregate data 
    if year in years:
        data = stats[year]
    elif isinstance(year, str) and year.upper() == 'ALL':
        data = total
    else:
        raise KeyError()
    
    #merge fantasy expert rankings with stats 
    data = df.merge(data[['Age'] + categories], on="PLAYER NAME")
    #filters by positions if inputted 
    if position.upper() in ["RB", "WR", "QB", "TE"]:
        filter = data["POS"].str.contains(position.upper())
        data = data.loc[filter]
    
    return data.sort_values(by="PPR", ascending=False)

display(get_stats(position="rb").head(12))

,RK,TEAM,POS,Age,G,GS,PassYds,PassTD,Int,RushAtt,RushYds,Y/A,RushTD,Tgt,Rec,RecYds,RecTD,FL,TD,PPR,VBD
PLAYER NAME,,,,,,,,,,,,,,,,,,,,,
Derrick Henry,5,TEN,RB4,27.0,13.333333,13.333333,0.21,0.043333,0.000000,24.866667,123.500000,5.000000,1.123333,2.126667,1.543333,11.186667,0.000000,0.086667,1.123333,21.933333,8.833333
Alvin Kamara,23,NO,RB13,26.0,14.071429,10.000000,0.00,0.000000,0.000000,15.285714,65.350000,4.404286,0.717143,6.210714,4.643214,42.692857,0.353214,0.000000,1.070357,21.903571,7.285714
Christian McCaffrey,1,SF,RB1,25.0,5.800000,5.800000,0.00,0.000000,0.000000,15.780000,66.670000,4.265000,0.599000,6.001000,5.404000,49.210000,0.197000,0.000000,0.803000,21.770000,0.000000
Dalvin Cook,14,MIN,RB7,26.0,13.518519,13.518519,0.00,0.000000,0.000000,20.807407,100.607407,4.826296,0.812593,3.816667,2.889630,21.659259,0.036296,0.181111,0.848889,20.151852,6.991111
Austin Ekeler,2,LAC,RB2,26.0,13.692308,13.692308,0.00,0.000000,0.000000,12.400000,55.400000,4.477692,0.500000,6.118462,4.772308,40.361538,0.384615,0.116923,0.884615,19.576923,4.615385
Najee Harris,42,PIT,RB20,23.0,17.000000,17.000000,0.00,0.000000,0.000000,18.100000,70.600000,3.910000,0.410000,5.530000,4.350000,27.500000,0.180000,0.000000,0.590000,17.700000,4.470000
Joe Mixon,27,CIN,RB14,25.0,13.272727,13.272727,0.00,0.000000,0.000000,18.709091,74.209091,3.985455,0.725455,3.362727,2.867273,20.527273,0.184545,0.090000,0.910000,17.618182,4.276364
Aaron Jones,22,GB,RB12,27.0,14.517241,14.517241,0.00,0.000000,0.000000,12.848276,65.658621,5.065862,0.448621,4.412069,3.416897,25.762069,0.274483,0.036207,0.727931,16.844828,3.827241
David Montgomery,35,CHI,RB17,24.0,14.071429,13.535714,0.00,0.000000,0.037143,16.871429,68.514286,4.070000,0.534643,4.246786,3.428214,26.414286,0.069643,0.074643,0.609643,16.446429,3.143571
